In [51]:
import pandas as pd
import datetime
import numpy as np

In [52]:
from transplant.config import (PATH_STATIC_CLEAN, PATH_DYNAMIC_CLEAN, 
DYNAMIC_CATEGORIES, DYNAMIC_HEADERS, STATIC_CATEGORIES)

In [53]:
dynamic = pd.read_csv(PATH_DYNAMIC_CLEAN, parse_dates=['time'])
static = pd.read_csv(PATH_STATIC_CLEAN, parse_dates=['date_sortie_bloc'])

Rapide notebook pour comprendre les différences entre le max date renvoyé par les instruments de mesure et l'heure de transfert au bloc. 

In [54]:
# Je trouve 3 dates de sortie de bloc NA pour les patients. 
# On prend l'heure de transfert en réa comme heure de fin de l'opération

static = static[static.date_sortie_bloc.isna() == False][['id_patient', 'date_sortie_bloc', 'heure_transfert_rea']]
static.columns = ['id_patient', 'date_sortie_bloc', 'time']

In [55]:
# On applique la même correction de date que pour date_transplantation

def correct_date_shift(x):
    idx_0h = (x['time'] - x['time'].shift(1)) < timedelta(days=0)  # 0h indexes
    for i in idx_0h.index[idx_0h]:  # loop over all 0h index
        x['time'].loc[i:] = x['time'].loc[i:] + timedelta(days=1)
    return x

static['time_sortie'] = pd.to_datetime(
        static['date_sortie_bloc'].dt.strftime('%Y-%m-%d') + ' ' + static['time'])
static = static.groupby('id_patient').apply(correct_date_shift)
dynamic_max = dynamic.groupby('id_patient').apply(max_ds)

In [66]:
def max_ds(df):
    return df.time.max()

dynamic_max = dynamic_max.reset_index()
dynamic_max.columns = ['id_patient', 'max_dynamic_date']

In [68]:
# On calcule la différence entre le max de la dynamque et la sortie du bloc. 

df = pd.merge(static, dynamic_max, on='id_patient')
df['diff'] = df.max_dynamic_date - df.time_sortie

In [80]:
df['diff'].describe()

count                           353
mean      3080 days 06:04:20.566572
std      10939 days 19:17:58.965075
min               -6 days +01:36:00
25%               -1 days +23:49:00
50%               -1 days +23:59:00
75%                 0 days 00:23:00
max             42709 days 17:57:00
Name: diff, dtype: object

In [85]:
# On trouve 26 valeurs anormales 

df[df['diff'] > pd.Timedelta('100 days')]


,id_patient,date_sortie_bloc,time,time_sortie,max_dynamic_date,diff
20,23,1900-01-01,10:00:00,1900-01-01 10:00:00,2012-04-27 09:56:00,41023 days 23:56:00
21,24,1900-01-02,10:30:00,1900-01-02 10:30:00,2012-04-30 10:25:00,41025 days 23:55:00
44,48,1900-01-01,05:03:00,1900-01-01 05:03:00,2012-11-13 05:03:00,41224 days 00:00:00
60,64,1900-01-02,01:15:00,1900-01-02 01:15:00,2013-02-11 01:06:00,41312 days 23:51:00
63,67,1900-01-01,04:45:00,1900-01-01 04:45:00,2013-02-27 04:45:00,41330 days 00:00:00
79,84,1900-01-02,02:30:00,1900-01-02 02:30:00,2013-05-04 02:20:00,41394 days 23:50:00
80,85,1900-01-01,02:40:00,1900-01-01 02:40:00,2013-05-07 02:49:00,41399 days 00:09:00
102,108,1900-01-02,06:35:00,1900-01-02 06:35:00,2013-10-15 05:59:00,41558 days 23:24:00
107,113,1900-01-01,05:00:00,1900-01-01 05:00:00,2013-11-09 20:06:00,41585 days 15:06:00
109,115,1900-01-01,04:22:00,1900-01-01 04:22:00,2013-11-19 04:20:00,41594 days 23:58:00
